In [4]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/u/shuhan/projects/vla')

from src.environments.highway_env.dataset import HighwayDataset, collate_fn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import torch
from torch.utils.data import DataLoader

In [5]:
from torch.utils.data import DataLoader

data_folder = '/storage/Datasets/highway_env/highway_fast_v0_dqn_meta_action/rollouts'
dataset = HighwayDataset(data_folder)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [6]:
len(dataset)

160000

In [14]:
for batch in dataloader:
  break


In [17]:
obs, act, valid_mask = batch
print(obs.shape, act.shape, valid_mask.shape)

torch.Size([32, 30, 5, 5]) torch.Size([32, 30]) torch.Size([32, 30])


In [32]:
(obs.flatten(2, 3) == obs.reshape(32, 30, -1)).all()
# load GPT-2 

tensor(True)

In [23]:
# load GPT-2 
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input, output_hidden_states=True)


In [38]:
import torch

batch_size = 2
seq_length = 10
hidden_size = model.config.hidden_size

embeddings = torch.randn(batch_size, seq_length, hidden_size)
attention_mask = torch.ones(batch_size, seq_length)

# Forward pass
outputs = model(inputs_embeds=embeddings, attention_mask=attention_mask, output_hidden_states=True)

In [37]:
outputs

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[-62.8273, -61.7893, -66.7804,  ..., -67.5077, -65.7401, -61.2149],
         [-68.2850, -68.2278, -70.4888,  ..., -75.0975, -75.1714, -67.2470],
         [-71.6926, -69.1768, -70.6928,  ..., -79.0380, -76.9401, -67.5034],
         ...,
         [-71.8272, -69.8833, -75.5365,  ..., -80.6712, -79.7190, -71.5455],
         [-67.0119, -64.4705, -69.8797,  ..., -71.8757, -69.4097, -64.7238],
         [-49.9565, -49.2613, -54.2038,  ..., -59.3608, -54.3507, -49.1795]],

        [[-66.1855, -64.8751, -69.1740,  ..., -71.6553, -71.5486, -65.7463],
         [-87.5645, -87.9511, -92.8867,  ..., -95.0146, -95.7339, -90.0031],
         [-71.8033, -71.8933, -78.4917,  ..., -80.3647, -78.5442, -73.4020],
         ...,
         [-69.3377, -69.0283, -72.8124,  ..., -75.2883, -75.8265, -72.9284],
         [-58.9582, -57.6775, -61.7406,  ..., -65.7993, -65.0396, -59.6103],
         [-66.9174, -65.4732, -68.5157,  ..., -71.8595, -71.0094, -65.8

In [25]:
output.hidden_states[-1].shape

torch.Size([1, 10, 768])

In [70]:
from src.models.vlas.cont_obs_token_action import ContObsTokenActionVLA

llm_backbone = model
hidden_dim = 768
obs_dim = 25
num_actions = 5
mlp_layers = 2

vla = ContObsTokenActionVLA(llm_backbone, obs_dim, num_actions, hidden_dim, mlp_layers)

In [71]:
outputs, loss_dict = vla(obs, act, valid_mask)

In [72]:
loss_dict

{'action': tensor(1.6000, grad_fn=<DivBackward0>),
 'obs': tensor(0.8963, grad_fn=<DivBackward0>),
 'reconst': tensor(0.2941, grad_fn=<DivBackward0>),
 'total': tensor(2.7904, grad_fn=<AddBackward0>)}

In [73]:
loss_dict['total'].backward()